In [ ]:
from typing import Literal
import pandas as pd
import folium
import geopandas as gpd

In [ ]:
PROBLEMA: Literal["P1", "P2", "P3"] = "P3"

In [ ]:
# 0 - Ler arquivos com os centros dos estados dos EUA
df_demand_centers = pd.read_csv(
    "../distances/states_centers.csv",  # Isso aqui representa cada demand center
    sep=",",
)
df_demand_centers.head()

In [ ]:
# Ler o arquivo com coordenadas dos Armazéns e Fabricas
df_warehouses = pd.read_excel(
    "../misc/tabelas.xlsx",
    sheet_name="table1",
)

df_plants = (
    df_warehouses.copy()
    .drop(
        # drop all rows after the first 4
        index=list(range(4, len(df_warehouses))),
    )
    .reset_index(drop=True)
)
if PROBLEMA == "P3":
    # add two new rows to the dataframe
    duas_novas_plantas = df_warehouses[
        df_warehouses["Location (city)"].isin(["Memphis *", "Chicago *"])
    ].reset_index(drop=True)
    df_plants = pd.concat(
        [
            df_plants,
            duas_novas_plantas,
        ],
        ignore_index=True,
    ).reset_index(drop=True)


display(df_plants)

# Dop first 4 rows
df_warehouses = df_warehouses.drop(index=[0, 1, 2, 3]).reset_index(drop=True)

if PROBLEMA == "P1":
    # Drop rows after index 18
    df_warehouses = df_warehouses.drop(
        index=list(range(18, len(df_warehouses)))
    ).reset_index(drop=True)
elif PROBLEMA == "P2":
    df_warehouses = df_warehouses[~df_warehouses["Location (city)"].isin(["New York*"])]
    pass
else:
    # Pro problema 3, remover Memphis e Chicago
    df_warehouses = df_warehouses[
        ~df_warehouses["Location (city)"].isin(
            ["Memphis *", "Chicago *", "Chicago", "Memphis", "New York*"]
        )
    ].reset_index(drop=True)


display(df_warehouses)

In [ ]:
# 1 - Plotar um mapa com os estados dos Estados Unidos (Usar folium e talvez geopandas), quero visualizar no jupyter notebook mesmo


def plot_map():
    # Download the US states GeoJSON file
    usa = gpd.read_file(
        "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
    )

    # Create a folium map centered on the US
    m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

    # Add the states to the map
    folium.GeoJson(
        usa,
        name="US States",
        style_function=lambda feature: {
            "fillColor": "#ffff00",
            "color": "black",
            "weight": 1,
            "fillOpacity": 0.2,
        },
        tooltip=folium.GeoJsonTooltip(fields=["name"], aliases=["State:"]),
    ).add_to(m)

    # Plot the state (i.e. demand) centers
    # Create a FeatureGroup for demand centers
    demand_layer = folium.FeatureGroup(name="Demand Centers")
    for _, row in df_demand_centers.iterrows():
        folium.Marker(
            location=[row["LATITUDE"], row["LONGITUDE"]],
            tooltip=row["ST2"],
            icon=folium.Icon(color="blue", icon="store", prefix="fa"),
        ).add_to(demand_layer)

    # Add the demand centers layer to the map
    demand_layer.add_to(m)

    # Plot the plants
    plants_layer = folium.FeatureGroup(name="Plants")
    for _, row in df_plants.iterrows():
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            tooltip=row["Location (city)"] + f" ({row['Location (state)']})",
            icon=folium.Icon(color="red", icon="industry", prefix="fa"),
        ).add_to(plants_layer)
    plants_layer.add_to(m)

    # Plot the warehouses
    warehouses_layer = folium.FeatureGroup(name="Warehouses")
    for _, row in df_warehouses.iterrows():
        folium.Marker(
            location=[row["Latitude"], row["Longitude"]],
            tooltip=row["Location (city)"] + f" ({row['Location (state)']})",
            icon=folium.Icon(color="green", icon="warehouse", prefix="fa"),
        ).add_to(warehouses_layer)
    warehouses_layer.add_to(m)

    # Add a layer selector to toggle the demand centers layer
    folium.LayerControl().add_to(m)

    # Return the map to display it in the notebook
    return m


# plot_map()

## Alternativa com cartopy

In [ ]:
# # Ler shapefile dos estados dos EUA
# usa = gpd.read_file(
#     "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data/us-states.json"
# )
# usa = usa[~usa["name"].isin(["Alaska", "Hawaii"])]

### Mapa 1

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib_scalebar.scalebar import ScaleBar

# set matplotlib as svg
plt.rcParams["figure.dpi"] = 300


# CRS de projeção (ex: Albers Equal Area para os EUA)
# Simples e geograficamente orientado
proj = ccrs.PlateCarree()

# Ou mais fiel aos mapas oficiais dos EUA
proj = ccrs.LambertConformal(
    central_longitude=-96, central_latitude=39, standard_parallels=(33, 45)
)


fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={"projection": proj})

# Plotar estados
usa.to_crs(proj.proj4_init).plot(
    ax=ax, edgecolor="black", facecolor="lightyellow", alpha=0.5
)

# Adicionar features naturais
ax.add_feature(cfeature.BORDERS, alpha=0.5)
ax.add_feature(cfeature.COASTLINE, alpha=0.7)
ax.add_feature(cfeature.LAKES, alpha=0.20)
ax.add_feature(cfeature.OCEAN, alpha=0.20)
# ax.add_feature(cfeature.RIVERS)

# Adicionar pontos dos demand centers
# ax.scatter(
#     df_demand_centers["LONGITUDE"],
#     df_demand_centers["LATITUDE"],
#     color="blue",
#     label="Demand Centers",
#     transform=ccrs.PlateCarree(),
#     marker="o",  # circle
#     s=80,
#     alpha=1,
# )

# Adicionar pontos das plantas
ax.scatter(
    df_plants["Longitude"],
    df_plants["Latitude"],
    color="red",
    label="Plants",
    transform=ccrs.PlateCarree(),
    marker="^",  # triangle_up
    s=80,
    alpha=1,
    zorder=20,
)

# Adicionar pontos dos armazéns
ax.scatter(
    df_warehouses["Longitude"],
    df_warehouses["Latitude"],
    color="green",
    label="Candidate Warehouses",
    transform=ccrs.PlateCarree(),
    marker="s",  # square
    s=80,
    alpha=1,
    zorder=10,
)

# Legenda, título
plt.legend(
    loc="upper left",
)
# plt.title("Distribuição de Infraestrutura nos EUA", fontsize=14)

# Adicionar escala gráfica (# 0.0001 pixel = 1 km)
scalebar = ScaleBar(0.0001, units="km", location="lower left")
ax.add_artist(scalebar)

# Adicionar rosa dos ventos
ax.text(
    0.95,
    0.20,
    "N",
    transform=ax.transAxes,
    fontsize=14,
    ha="center",
    va="center",
    fontweight="bold",
)
ax.annotate(
    "",
    xy=(0.95, 0.20 + 0.06),
    xytext=(0.95, 0.20 + 0.02),
    arrowprops=dict(facecolor="black", width=2, headwidth=8),
    xycoords="axes fraction",
)

# Salvar como PNG
plt.savefig(f"mapa_1_problema_{PROBLEMA[1]}.png", dpi=300)  # , bbox_inches="tight")
plt.show()

In [ ]:
# TODO:
# 1 - Mapa com armazéns candidatos cenário 1, 2 e 3 (OK)
# 2 - Mapa com quais armazéns foram abertos nos cenário 1, 2 e 3
# 3 -
# 4 -


In [ ]:
df_warehouses
#         "Atlanta"
#         "Detroit"
#         "Baltimore"
#         "Orlando"
#         "Portland"
#         "Denver"
#         "Camp Hill"
#         "Houston"
#         "Las Vegas"
#         "Minneapolis"
#         "Phoenix"
#         "Richmond"
#         "Salt Lake City"
#         "Hartford"


#         "Memphis *"
#         "Chicago *",

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from matplotlib_scalebar.scalebar import ScaleBar

# set matplotlib as svg
plt.rcParams["figure.dpi"] = 300


# CRS de projeção (ex: Albers Equal Area para os EUA)
# Simples e geograficamente orientado
proj = ccrs.PlateCarree()

# Ou mais fiel aos mapas oficiais dos EUA
proj = ccrs.LambertConformal(
    central_longitude=-96, central_latitude=39, standard_parallels=(33, 45)
)


fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={"projection": proj})

# Plotar estados
usa.to_crs(proj.proj4_init).plot(
    ax=ax, edgecolor="black", facecolor="lightyellow", alpha=0.5
)

# Adicionar features naturais
ax.add_feature(cfeature.BORDERS, alpha=0.5)
ax.add_feature(cfeature.COASTLINE, alpha=0.7)
ax.add_feature(cfeature.LAKES, alpha=0.20)
ax.add_feature(cfeature.OCEAN, alpha=0.20)
# ax.add_feature(cfeature.RIVERS)

# Adicionar pontos dos demand centers
# ax.scatter(
#     df_demand_centers["LONGITUDE"],
#     df_demand_centers["LATITUDE"],
#     color="blue",
#     label="Demand Centers",
#     transform=ccrs.PlateCarree(),
#     marker="o",  # circle
#     s=80,
#     alpha=1,
# )

################################################################################
# Filtros Malditos

open_color = "green"
closed_color = "gray"
open_plant_color = "red"
closed_plant_color = "black"

df_warehouses["is_open"] = False
df_plants["is_open"] = True

match PROBLEMA:
    case "P1":
        # definir quais warehouses estão abertos ou não.
        for i, row in df_warehouses.iterrows():
            if row["Location (city)"] in [
                "Atlanta",
                "Boston",
                "Davenport",
                "Detroit",
                "Greensboro",
                "Kansas City",
                "Baltimore",
                "Memphis",
                "Milwaukee",
                "Orlando",
                "Portland",
            ]:
                # set is_open to True
                df_warehouses.at[i, "is_open"] = True

    case "P2":
        # definir quais warehouses estão abertos ou não.
        # NOTE: por algum motivo não funcionou, crazy.
        # for i, row in df_warehouses.iterrows():
        #     print(f'"{row["Location (city)"]}"')
        #     if row["Location (city)"] in [
        #         # "Covington",
        #         # "New York",
        #         # "Arlington",
        #         # "Long Beach",
        #         "Atlanta"
        #         "Detroit"
        #         "Baltimore"
        #         "Orlando"
        #         "Portland"
        #         "Denver"
        #         "Camp Hill"
        #         "Houston"
        #         "Las Vegas"
        #         "Minneapolis"
        #         "Phoenix"
        #         "Richmond"
        #         "Salt Lake City"
        #         "Hartford"
        #         "Memphis *"
        #         "Chicago *",
        #     ]:
        #         # set is_open to True
        #         print(row["Location (city)"])
        for i in [0, 6, 10, 13, 15, 20, 22, 23, 24, 25, 27, 28, 30, 39, 42, 43]:
            df_warehouses.at[i, "is_open"] = True

    case "P3":
        # definir quais warehouses estão abertos ou não.
        # definir quais
        for i, row in df_warehouses.iterrows():
            if row["Location (city)"] in [
                "Covington",
                "New York",
                "Arlington",
                "Long Beach",
                "Memphis",
                "Chicago",
                "Atlanta",
                "Boston",
                "Davenport",
                "Kansas City",
                "Orlando",
                "Portland",
                "Billings",
                "Denver",
                "Camp Hill",
                "Las Vegas",
                "Minneapolis",
                "Phoenix",
                "Salt Lake City",
            ]:
                # set is_open to True
                df_warehouses.at[i, "is_open"] = True

        # Somente Memphis não foi aberta
        df_plants["is_open"].iloc[4] = False


# Adicionar pontos das plantas
df_open_plants = df_plants[df_plants["is_open"] == True]
df_closed_plants = df_plants[df_plants["is_open"] == False]
ax.scatter(
    df_open_plants["Longitude"],
    df_open_plants["Latitude"],
    color=open_plant_color,
    label="Open Plants",
    transform=ccrs.PlateCarree(),
    marker="^",  # triangle_up
    s=80,
    alpha=1,
    zorder=20,
)
ax.scatter(
    df_closed_plants["Longitude"],
    df_closed_plants["Latitude"],
    color=closed_plant_color,
    label="Closed Plants",
    transform=ccrs.PlateCarree(),
    marker="^",  # triangle_up
    s=80,
    alpha=1,
    zorder=20,
)

# Adicionar pontos dos armazéns
df_open = df_warehouses[df_warehouses["is_open"] == True]
ax.scatter(
    df_open["Longitude"],
    df_open["Latitude"],
    color=open_color,
    label="Open Warehouses",
    transform=ccrs.PlateCarree(),
    marker="s",  # square
    s=80,
    alpha=1,
    zorder=2,
)
df_closed = df_warehouses[df_warehouses["is_open"] == False]
ax.scatter(
    df_closed["Longitude"],
    df_closed["Latitude"],
    color=closed_color,
    label="Closed Warehouses",
    transform=ccrs.PlateCarree(),
    marker="s",  # square
    s=80,
    alpha=1,
    zorder=2,
)


# Legenda, título
plt.legend(
    loc="upper left",
)
# plt.title("Distribuição de Infraestrutura nos EUA", fontsize=14)

# Adicionar escala gráfica (# 0.0001 pixel = 1 km)
scalebar = ScaleBar(0.0001, units="km", location="lower left")
ax.add_artist(scalebar)


# Adicionar rosa dos ventos
ax.text(
    0.95,
    0.20,
    "N",
    transform=ax.transAxes,
    fontsize=14,
    ha="center",
    va="center",
    fontweight="bold",
)
ax.annotate(
    "",
    xy=(0.95, 0.20 + 0.06),
    xytext=(0.95, 0.20 + 0.02),
    arrowprops=dict(facecolor="black", width=2, headwidth=8),
    xycoords="axes fraction",
)

# Salvar como PNG
plt.savefig(f"mapa_2_problema_{PROBLEMA[1]}.png", dpi=300)  # , bbox_inches="tight")
plt.show()

In [ ]:
df_warehouses